In [ ]:
#| default_exp eval.metrics

In [ ]:
#| export

from lapsolver import solve_dense
import numpy as np
import torch

In [ ]:
#| export

def magnitude_spectrum(x):
    return torch.abs(torch.fft.rfft(x, norm="forward"))

def power_spectrum(x):
    return magnitude_spectrum(x) ** 2

def log_spectral_distance(x, y):
    return torch.sqrt(torch.mean((10 * torch.log10(power_spectrum(x) / power_spectrum(y))) ** 2))

def spectral_mse_db(x, y):
    return torch.mean(20 * torch.log10(magnitude_spectrum(x) / magnitude_spectrum(y)))

def spectral_mse(x, y):
    return torch.mean((magnitude_spectrum(x) - magnitude_spectrum(y)) ** 2)

def itakura_saito(x, y):
    P_x = power_spectrum(x)
    P_y = power_spectrum(y)
    return torch.mean(P_x / P_y - torch.log(P_x / P_y))

In [ ]:
# | export


def single_sinusoid_metrics(
    target_signal: torch.Tensor,
    target_freq: torch.Tensor,
    target_amp: torch.Tensor,
    target_snr: torch.Tensor,
    predicted_signal: torch.Tensor,
    predicted_freq: torch.Tensor,
    predicted_amp: torch.Tensor,
):
    """Calculate metrics for single sinusoid evaluation."""

    data = dict(
        target_freq=target_freq.cpu().numpy(),
        target_amp=target_amp.cpu().numpy(),
        target_snr=target_snr.cpu().numpy(),
        predicted_freq=predicted_freq.cpu().numpy(),
        predicted_amp=predicted_amp.cpu().numpy(),
    )

    # Calculate metrics

    data["freq_mse"] = ((target_freq - predicted_freq) ** 2).cpu().numpy()
    data["freq_mse_db"] = 10 * np.log10(data["freq_mse"])
    data["amp_mse"] = ((target_amp - predicted_amp) ** 2).cpu().numpy()
    data["amp_mse_db"] = 10 * np.log10(data["amp_mse"])
    data["signal_mse"] = torch.mean((target_signal - predicted_signal) ** 2, dim=-1).cpu().numpy()
    data["signal_mse_db"] = 10 * np.log10(data["signal_mse"])

    joint_target = torch.stack((target_freq, target_amp), dim=-1)
    joint_predicted = torch.stack((predicted_freq, predicted_amp), dim=-1)
    data["joint_mse"] = torch.mean((joint_target - joint_predicted) ** 2, dim=-1).cpu().numpy()
    data["joint_mse_db"] = 10 * np.log10(data["joint_mse"])

    return data

In [ ]:
#| export
def min_lap_cost(target, predicted, unsqueeze=False):
    """Calculate the minimum lap cost between two sets of points."""
    if unsqueeze:
        target = target[..., None]
        predicted = predicted[..., None]

    cost = (torch.cdist(target, predicted) ** 2).cpu().numpy()
    costs = []
    for i in range(cost.shape[0]):
        r, c = solve_dense(cost[i])
        costs.append(cost[i, r, c].mean())
    return costs

In [ ]:
#| export

def chamfer_distance(target, predicted, unsqueeze=False):
    """Calculate the Chamfer distance between two sets of points."""
    if unsqueeze:
        target = target[..., None]
        predicted = predicted[..., None]

    # Compute distance matrix
    distances = torch.cdist(target, predicted) ** 2

    # Compute the minimum distance across both target and predicted dimensions
    target_min_distances = torch.min(distances, dim=-1).values
    predicted_min_distances = torch.min(distances, dim=-2).values

    # Compute the mean of the minimum distances
    target_mean_min_distance = torch.mean(target_min_distances, dim=-1)

    # Compute the mean of the minimum distances
    predicted_mean_min_distance = torch.mean(predicted_min_distances, dim=-1)

    return target_mean_min_distance + predicted_mean_min_distance

In [ ]:
a = torch.randn(16, 3)
b = torch.randn(16, 3)

chamfer_distance(a, b, unsqueeze=True).shape

torch.Size([16])

In [ ]:
# | export


def multi_sinusoid_metrics(
    target_signal: torch.Tensor,
    target_freq: torch.Tensor,
    target_amp: torch.Tensor,
    target_snr: torch.Tensor,
    predicted_signal: torch.Tensor,
    predicted_freq: torch.Tensor,
    predicted_amp: torch.Tensor,
):
    """Calculate metrics for single sinusoid evaluation."""

    data = dict(
        target_freq=[t.numpy() for t in target_freq.cpu().split(1)],
        target_amp=[t.numpy() for t in target_amp.cpu().split(1)],
        target_snr=target_snr.cpu().numpy(),
        predicted_freq=[t.numpy() for t in predicted_freq.cpu().split(1)],
        predicted_amp=[t.numpy() for t in predicted_amp.cpu().split(1)],
    )

    # Calculate metrics
    # NOTE: we opted not to use the linear assignment cost or Chamfer distance as
    # metrics in the final experiments, as they appear to be poorly correlated with
    # perceptual similarity. We leave them here for completeness, and will likely
    # revisit them in the future work.

    data["freq_lac"] = min_lap_cost(target_freq, predicted_freq, unsqueeze=True)
    data["freq_lac_db"] = 10 * np.log10(data["freq_lac"])
    data["amp_lac"] = min_lap_cost(target_amp, predicted_amp, unsqueeze=True)
    data["amp_lac_db"] = 10 * np.log10(data["amp_lac"])
    data["signal_mse"] = (
        torch.mean((target_signal - predicted_signal) ** 2, dim=-1).cpu().numpy()
    )
    data["signal_mse_db"] = 10 * np.log10(data["signal_mse"])
    data["freq_chamfer"] = (
        chamfer_distance(target_freq, predicted_freq, unsqueeze=True).cpu().numpy()
    )
    data["amp_chamfer"] = (
        chamfer_distance(target_amp, predicted_amp, unsqueeze=True).cpu().numpy()
    )
    joint_target = torch.stack((target_freq, target_amp), dim=-1)
    joint_predicted = torch.stack((predicted_freq, predicted_amp), dim=-1)
    data["joint_lac"] = min_lap_cost(joint_target, joint_predicted)
    data["joint_lac_db"] = 10 * np.log10(data["joint_lac"])
    data["joint_chamfer"] = (
        chamfer_distance(joint_target, joint_predicted).cpu().numpy()
    )

    data["spectral_mse"] = 10 * np.log10(
        np.stack(
            list(
                map(
                    spectral_mse,
                    target_signal.cpu().split(1),
                    predicted_signal.cpu().split(1),
                )
            ),
            axis=0,
        )
    )

    return data